# Generating backgrounds

In [ ]:
%matplotlib inline

In [ ]:
from pathlib import Path

import matplotlib as mpl

from thesis_cover import *

mpl.rcParams["figure.dpi"] = 300

options = sorted(Path("data").glob("*/*.pickle"))


def generate_cover(
    learner,
    save_fname: str,
    npoints_interp=1000,
    dpi=300,
    resolution=(4096, 2160),
    cmap=None,
):
    data = list(learner.data.items())
    x_size, y_size = xy_size = (resolution[0] / dpi, resolution[1] / dpi)
    fig, ax = plt.subplots(figsize=(x_size, y_size))
    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)

    ax.set_xticks([])
    ax.set_yticks([])

    cmap = cmap or get_cmap("inferno", 0.15, 0.95, 1.15)
    npoints_tri = len(data) // 4
    if len(data) > 4000:
        npoints_tri = max(npoints_tri, 4000)

    im, line1, line2 = get_new_artists(
        npoints_tri, learner, data, ax, xy_size, npoints_interp, cmap
    )

    ax.axis("off")

    ax.set_xlim(-x_size / 2, x_size / 2)
    ax.set_ylim(-y_size / 2, y_size / 2)
    print(f"Saving {save_fname}")
    if save_fname is not None:
        fig.savefig(
            save_fname, format=save_fname.suffix[1:], pad_inches=0, dpi=dpi,
        )
    else:
        plt.show()
    plt.close(fig)


folder = Path("background-defense")
folder.mkdir(exist_ok=True)

## Pick one using a widget

In [ ]:
import adaptive
from ipywidgets import Dropdown, IntText, interact


def get_background(fname, cmap, x_resolution, y_resolution):
    learner = load_learner(fname)
    cmap = get_cmap(cmap, 0.10, 0.85, 0.85)
    generate_cover(
        learner,
        None,
        cmap=cmap,
        npoints_interp=2000,
        resolution=(x_resolution, y_resolution),
    )


interact(
    get_background,
    fname=Dropdown(options=options),
    cmap=Dropdown(options=plt.colormaps(), value="inferno"),
    x_resolution=IntText(value=4096),
    y_resolution=IntText(value=2160),
)

## Generate all

In [ ]:
for i, fname in enumerate(options):
    learner = load_learner(fname)
    print(f"cover {i+1}, npoints: {learner.npoints}")
    cmap = get_cmap("inferno", 0.10, 0.85, 0.85)
    generate_cover(
        learner,
        fname_out(folder, fname).with_suffix(".png"),
        cmap=cmap,
        npoints_interp=2000,
        dpi=300,
    )
    print()